In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

from pyphocorehelpers.print_helpers import generate_html_string

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions
import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_expected_vs_observed_firing_rates
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import _update_pipeline_missing_preprocessing_parameters
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

good_contexts_list = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54')
]

from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass




In [3]:
%pdb off
# %%viztracer
# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE


local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
# saving_mode = PipelineSavingScheme.SKIP_SAVING
# force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
force_reload = True


## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.


    # ==================================================================================================================== #
    # Load Pipeline                                                                                                        #
    # ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s


Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15
Skipping loading from pickled file because force_reload == True.
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.epochs_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.position_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.spikes.mat... 

C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\session\Formats\SessionSpecifications.py:140: UserWarning: WARNING: Optional File: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


done.
Failure loading .position.npy. Must recompute.

Saving updated position results results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.position.npy... 2006-6-08_14-26-15.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
	 Saving updated interpolated spike position results results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.interpolated_spike_positions.npy... 2006-6-08_14-26-15.interpolated_spike_positions.npy saved
done.
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.laps_info.mat... done.
setting laps object.
session.laps loaded successfully!
Loading matlab import file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\2006-6-08_14-26-15.replay_info.mat... done.
session.replays loaded successfully!
Loading success: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-

C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:70: SettingWithCopyWarning: 
A value is trying to b

Applying session filter named "maze2"...
Constraining to epoch with times (start: 1211.5580800310709, end: 2093.8978568242164)
computing neurons mua for session...



...

Applying session filter named "maze"...
Constraining to epoch with times (start: 0.0, end: 2093.8978568242164)
computing neurons mua for session...



C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:70: SettingWithCopyWarning: 
A value is trying to b

using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (25.5637332724328, 257.964172947664)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((25.5637332724328, 257.964172947664), (89.1844223602494, 131.92462510535915))
	 done.
	 spikes_df[time_variable_name]: (18947,) should be less than time_window_edges: (35889,)!
	 spikes_df[time_variable_name]: (18947,) should be less than time_window_edges: (35889,)!


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\Analysis\Decoder\reconstruction.py:363: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (36246,)
updating computation_results...
done.
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (25.5637332724328, 257.964172947664)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((25.5637332724328, 257.964172947664), (89.1844223602494, 131.92462510535915))
	 done.
	 spikes_df[time_variable_name]: (15487,) should be less than time_window_edges: (24848,)!
	 spikes_df[time_variable_name]: (15487,) should be less than time_window_edges: (24848,)!
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (25095,)
updating computation_results...
done.
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (25.5637332724328, 257.96417294

In [4]:
%matplotlib qt
from neuropy.utils.matplotlib_helpers import interactive_select_grid_bin_bounds_2D
# fig, ax, rect_selector, set_extents = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True)

grid_bin_bounds = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True, should_apply_updates_to_pipeline=False)
print(f'grid_bin_bounds: {grid_bin_bounds}')
print(f"Add this to `specific_session_override_dict`:\n\n{curr_active_pipeline.get_session_context().get_initialization_code_string()}:dict(grid_bin_bounds=({(grid_bin_bounds[0], grid_bin_bounds[1]), (grid_bin_bounds[2], grid_bin_bounds[3])})),\n")

blocking and waiting for user input. Press [enter] to confirm selection change or [esc] to revert with no change.
user confirmed extents: (25.5637332724328, 257.964172947664, 89.1844223602494, 131.92462510535915)
Add this to `specific_session_override_dict`:

IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):dict(grid_bin_bounds=(((25.5637332724328, 257.964172947664), (89.1844223602494, 131.92462510535915)))),

grid_bin_bounds: (25.5637332724328, 257.964172947664, 89.1844223602494, 131.92462510535915)
Add this to `specific_session_override_dict`:

IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'):dict(grid_bin_bounds=(((25.5637332724328, 257.964172947664), (89.1844223602494, 131.92462510535915)))),



In [5]:
## Post Compute Validate 2023-05-16:
from scripts.run_BatchAnalysis import post_compute_validate

post_compute_validate(curr_active_pipeline)

were pipeline preprocessing parameters missing and updated?: False


In [6]:
curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
force_recompute_global = force_reload
# force_recompute_global = True
# extended_computations_include_includelist=['long_short_post_decoding'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if len(newly_computed_values) > 0:
	print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
	try:
		# curr_active_pipeline.global_computation_results.persist_time = datetime.now()
		# Try to write out the global computation function results:
		curr_active_pipeline.save_global_computation_results()
	except Exception as e:
		print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
		print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
	print(f'no changes in global results.')
# 10m 29.5s

included includelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis missing.
	 Recomputing jonathan_firing_rate_analysis...
	 done.
long_short_fr_indicies_analyses missing.
	 Recomputing long_short_fr_indicies_analyses...
	 done.
long_short_decoding_analyses missing.
	 Recomputing long_short_decoding_analyses...
sane_midpoint_x: 142.21650629615837, hardcoded_track_midpoint_x: None, track_min_max_x: (22.736279243974774, 261.696733348342)
hardcoded_track_midpoint_x is None, falling back to sane_midpoint_x... 142.21650629615837
desc_crossings_x: (20,), asc_crossings_x: (20,)
setting new computation epochs because laps changed.
using self.config.grid_bin_bounds_1D: (25.5637332724328, 257.964172947664)
using self.config.grid_bin_bounds: ((25.563733

In [7]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)


https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/
https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-149
shared_aclus: [3, 5, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18, 20, 22, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 55, 56, 57, 59, 60, 61, 62, 64, 65, 66, 69, 71, 72, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]
long_only_aclus: [2, 8, 19, 28, 36, 54, 63, 68, 73, 74, 109]
short_only_aclus: [4, 13, 21, 23, 52, 58, 67, 70, 81]
shared_aclus: [3, 5, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18, 20, 22, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 55, 56, 57, 59, 60, 61, 62, 64, 65, 66, 69, 71, 72, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]
long_on

on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =90
	selection_indicies: [90]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121]])


Traceback (most recent call last):
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\matplotlib\cbook\__init__.py", line 304, in process
    func(*args, **kwargs)
  File "C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\Pho2D\stacked_epoch_slices.py", line 601, in on_selected_epochs_changed
    self.on_click(event=event)
  File "C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\GUI\Qt\Mixins\PaginationMixins.py", line 71, in on_click
    self.params.is_selected[found_data_index] = not self.params.is_selected.get(found_data_index, False) # if never set before, assume that it's not selected
TypeError: unhashable type: 'numpy.ndarray'


on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =11
	selection_indicies: [11]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121]])


Traceback (most recent call last):
  File "c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\.venv\lib\site-packages\matplotlib\cbook\__init__.py", line 304, in process
    func(*args, **kwargs)
  File "C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\Pho2D\stacked_epoch_slices.py", line 601, in on_selected_epochs_changed
    self.on_click(event=event)
  File "C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\GUI\Qt\Mixins\PaginationMixins.py", line 71, in on_click
    self.params.is_selected[found_data_index] = not self.params.is_selected.get(found_data_index, False) # if never set before, assume that it's not selected
TypeError: unhashable type: 'numpy.ndarray'


on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =9
	selection_indicies: [ 9 11]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =14
	selection_indicies: [ 9 11 14]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =15
	selection_indicies: [ 9 11 14 15]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =89
	selection_indicies: [89 90]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =28
	selection_indicies: [28 89 90]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =27
	selection_indicies: [27 28 89 90]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =25
	selection_indicies: [25 27 28 89 90]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =37
	selection_indicies: [25 27 28 37 89 90]
on_selected_epochs_changed(...)
current_page_idx = 0, found_data_index =45
	selection_indicies: [25 27 28 37 45 89 90]
on_selected

In [ ]:
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner
neptuner = None
if neptuner is None:
	neptuner = Neptuner.init_with_pipeline(curr_active_pipeline)
try:
	succeeded_fig_paths, failed_fig_paths = neptuner.upload_figures(curr_active_pipeline)
	# neptune_output_figures(curr_active_pipeline)
except Exception as e:
	print(f'in `_perform_plots(...)`: neptune_output_figures(...) failed with exception: {e}. Continuing.')
finally:
	neptuner.stop()
neptuner.stop()

### Pipeline Loading/Saving [Optional]

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object
# PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell
# TypeError: cannot pickle 'PyQt5.QtCore.pyqtSignal' object

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [ ]:
curr_active_pipeline.save_global_computation_results() # PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell

In [ ]:
curr_active_pipeline.load_pickled_global_computation_results()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_global_computation_results()

# Get output files:

In [ ]:
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatBaseRegisteredClass
from neuropy.core.session.KnownDataSessionTypeProperties import KnownDataSessionTypeProperties
from neuropy.core.session.dataSession import DataSession
from neuropy.core.session.Formats.SessionSpecifications import SessionFolderSpec, SessionFileSpec

# 'ripple_df.pkl', .ripple.npy'

# files_list = [curr_active_pipeline.sess.filePrefix.with_suffix(a_suffix) for a_suffix in session_folder_output_file_suffix_list]
# files_list

"""
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.interpolated_spike_positions.npy,"2023-06-07 01:03:36",35290406
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.mua.npy,"2023-06-07 01:03:37",7766554
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.pbe.npy,"2023-06-07 01:03:37",10902
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.position.npy,"2023-06-07 01:03:36",2446247
"""

session_name = curr_active_pipeline.session_name
print(f'session_name: {session_name}')
output_folder_spec: SessionFolderSpec = SessionFolderSpec(required=[
        SessionFileSpec('loadedSessPickle.pkl', session_name, 'The primary .pkl file after computing the session', None),
        SessionFileSpec('{}.interpolated_spike_positions.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.mua.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.pbe.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.position.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('output/global_computation_results.pkl', session_name, 'The global_computation_results computed for the pipeline.', None),
    ],
    optional=[
        SessionFileSpec('output/leave_one_out_results_short.pkl', session_name, 'Optional caching file.', None),
        SessionFileSpec('output/leave_one_out_surprise_results_long.pkl', session_name, 'Optional caching file', None),
        SessionFileSpec('ripple_df.pkl', session_name, 'SWR ripples computed by an external script from another repo.', None),
        SessionFileSpec('{}.ripple.npy', session_name, 'Ripples computed by this repo and cached to the directory.', None),
    ]
)


is_resolved, resolved_required_filespecs_dict, resolved_optional_filespecs_dict = output_folder_spec.validate(curr_active_pipeline.sess.basepath)

if is_resolved:
    print(f'resolved_required_filespecs_dict: {resolved_required_filespecs_dict}')
    

# out_dict = {}
# out_dict['required_files'] = [str(a_path) for a_path in resolved_required_filespecs_dict.keys() if a_path.resolve().exists()]
# out_dict['optional_files'] = [str(a_path) for a_path in resolved_optional_filespecs_dict.keys() if a_path.resolve().exists()]
# out_dict

# Get the list of strings for all output files that actually exist:
all_output_file_paths: list[str] = [str(a_path) for a_path in resolved_required_filespecs_dict.keys() if a_path.resolve().exists()] + [str(a_path) for a_path in resolved_optional_filespecs_dict.keys() if a_path.resolve().exists()]
all_output_file_paths


In [ ]:

# curr_active_pipeline.register_output_file(output_path=output_path, 
curr_active_pipeline.registered_output_files_list
curr_active_pipeline.registered_output_files
# curr_active_pipeline.get_output_path()

# reveal_in_system_file_manager(curr_active_pipeline.get_daily_programmatic_session_output_path())

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadData, saveData

# Save in case something goes wrong:
temp_out_path = 'registered_output_files.pkl'


In [ ]:
saveData(temp_out_path, (curr_active_pipeline.registered_output_files.to_dict()))
reveal_in_system_file_manager(temp_out_path)

In [ ]:
test_registered_output_files = loadData(temp_out_path)
test_registered_output_files
curr_active_pipeline.registered_output_files = test_registered_output_files

# End Run

In [8]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

# 2023-06-13 - Updating Object Persistance from the dates added to the local computations:
TODO 2023-06-13 18:07: - [ ] The global_computation_results should check the `current_computation_completion_times` corresponding to the local functions that contribute results that they depend on (like 'pf1D', etc) to make sure they haven't changed. If they have changed, they should recompute.

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import LoadedObjectPersistanceState
from datetime import datetime
# curr_active_pipeline.pipeline_compare_dict
# curr_active_pipeline.computation_results['maze1']['computation_times']
	
any_most_recent_computation_time, each_epoch_latest_computation_time, each_epoch_each_result_computation_completion_times = curr_active_pipeline.get_computation_times()
each_epoch_each_result_computation_completion_times

# any_latest_computation_time: 2023-06-13 17:40:44.628404
# current_epoch_latest_computation_time: {'maze1': datetime.datetime(2023, 6, 13, 17, 39, 18, 195870), 'maze2': datetime.datetime(2023, 6, 13, 17, 40, 44, 628404), 'maze': datetime.datetime(2023, 6, 13, 17, 33, 33, 859610)}
# current_computation_status: {'maze1': {'_perform_baseline_placefield_computation': datetime.datetime(2023, 6, 13, 17, 24, 17, 681574),
#   '_perform_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 39, 18, 195870),
#   '_perform_two_step_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 26, 17, 278340),
#   '_perform_pf_find_ratemap_peaks_computation': datetime.datetime(2023, 6, 13, 17, 26, 21, 5364),
#   '_perform_extended_statistics_computation': datetime.datetime(2023, 6, 13, 17, 26, 21, 31363),
#   '_perform_firing_rate_trends_computation': datetime.datetime(2023, 6, 13, 17, 26, 22, 385595)},
#  'maze2': {'_perform_baseline_placefield_computation': datetime.datetime(2023, 6, 13, 17, 26, 23, 665116),
#   '_perform_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 40, 44, 628404),
#   '_perform_two_step_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 28, 32, 360685),
#   '_perform_pf_find_ratemap_peaks_computation': datetime.datetime(2023, 6, 13, 17, 28, 36, 25350),
#   '_perform_extended_statistics_computation': datetime.datetime(2023, 6, 13, 17, 28, 36, 49351),
#   '_perform_firing_rate_trends_computation': datetime.datetime(2023, 6, 13, 17, 28, 37, 302445)},
#  'maze': {'_perform_baseline_placefield_computation': datetime.datetime(2023, 6, 13, 17, 28, 39, 869814),
#   '_perform_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 31, 56, 800175),
#   '_perform_two_step_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 33, 26, 857989),
#   '_perform_pf_find_ratemap_peaks_computation': datetime.datetime(2023, 6, 13, 17, 33, 30, 796529),
#   '_perform_extended_statistics_computation': datetime.datetime(2023, 6, 13, 17, 33, 30, 836528),
#   '_perform_firing_rate_trends_computation': datetime.datetime(2023, 6, 13, 17, 33, 33, 859610)}}



In [ ]:
curr_active_pipeline.active_completed_computation_result_names

In [ ]:
curr_active_pipeline.pipeline_compare_dict

In [ ]:

# ('computed_data', ('pf1D', 'pf2D', 'pf1D_Decoder', 'pf2D_Decoder', 'pf2D_TwoStepDecoder', 'RatemapPeaksAnalysis', 'extended_stats', 'firing_rate_trends'))


# 2023-05-26 - Expected vs Observed firing rates
## 2023-05-30 Comments from Kourosh:
- concern: is it valid to use the most-likely predicted position for each decoder for this? Isn't it a bit circular because the firing rates determine the likelihood of the position?
- sign-rank is equivalent of paired t-test for non-parametric, use that to compare the means for each cell after averaging over replays to remove the noise from that
- OPTION: could add a continuous plot that shows the difference between the two decoders and the animal's measured position as a function of time (observed_x - long_decoded_x), (observed_x - short_decoded_x).
- [x] Plot just replay-to-replay series, removing the time axis

In [ ]:
## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

num_epochs: int = len(num_timebins_in_epoch)

## 2023-05-30 - `plot_expected_vs_observed`

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _prepare_plot_expected_vs_observed

fig, axes, final_context, active_out_figure_paths = _prepare_plot_expected_vs_observed(curr_active_pipeline, defer_render=False)

In [ ]:
jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf

In [ ]:
jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins #.shape

In [ ]:
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
instantaneous_fr_df = deepcopy(jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.instantaneous_unit_specific_spike_rate_values)
instantaneous_fr_df['t'] = jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins
instantaneous_fr_df #.plot()

smoothed_instantaneous_fr_df = instantaneous_fr_df.ewm(span=60).mean()

smoothed_instantaneous_fr_df.plot(x='t')
plt.title("Instantaneous Firing Rates")
plt.xlabel('time')
plt.ylabel('Firing Rate (Hz)')

In [ ]:
instantaneous_fr_df.set_index('t').to_clipboard()

In [ ]:
## Require placefield presence on either the long or the short
jonathan_firing_rate_analysis_result.neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])]

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()

# 2023-05-19 - Testing S-only emergence, L-only replays in S, peak position remappings 

In [33]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()
neuron_replay_stats_df

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
2,212.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.305475,0.075055,-0.230420,10.361654,NaN,NaN,5.333564,NaN,NaN,0,26,26,0,NeuronType.PYRAMIDAL
3,144.563733,True,198.563733,True,False,SplitPartitionMembership.SHARED,0.257163,1.096455,0.839293,13.312611,9.673209,-3.639401,6.784887,5.384832,-1.400054,1,64,18,46,NeuronType.PYRAMIDAL
4,NaN,False,178.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,0.100774,0.760024,0.659250,NaN,14.188911,NaN,NaN,7.474468,NaN,2,61,0,61,NeuronType.PYRAMIDAL
5,98.563733,True,112.563733,True,False,SplitPartitionMembership.SHARED,1.873265,1.818093,-0.055172,12.866177,12.551413,-0.314764,7.369721,7.184753,-0.184968,3,134,70,64,NeuronType.PYRAMIDAL
6,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,27.990926,13.591147,-14.399779,115.355139,44.210620,-71.144520,71.673033,28.900883,-42.772149,4,360,198,162,NeuronType.CONTAMINATED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,0.129664,0.146140,0.016476,6.439696,7.921885,1.482188,3.284680,4.034012,0.749332,103,9,5,4,NeuronType.PYRAMIDAL
106,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,12.817478,13.180032,0.362554,47.537618,50.207624,2.670006,30.177548,31.693828,1.516280,104,359,196,163,NeuronType.CONTAMINATED
107,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,1.690138,1.061377,-0.628761,18.250585,18.163205,-0.087380,9.970361,9.612291,-0.358070,105,135,82,53,NeuronType.INTERNEURONS
108,238.563733,True,206.563733,True,False,SplitPartitionMembership.SHARED,2.524455,3.327796,0.803341,20.897480,20.870241,-0.027238,11.710967,12.099019,0.388051,106,243,119,124,NeuronType.PYRAMIDAL


In [34]:
neuron_replay_stats_df.sort_values(by=['long_pf_peak_x'], inplace=False, ascending=True)

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
102,36.563733,True,88.563733,True,False,SplitPartitionMembership.SHARED,1.667443,1.431349,-0.236095,7.706498,8.130911,0.424413,4.686970,4.781130,0.094159,100,39,20,19,NeuronType.PYRAMIDAL
87,36.563733,True,72.563733,True,False,SplitPartitionMembership.SHARED,0.711118,0.550887,-0.160231,8.432868,10.971056,2.538188,4.571993,5.760972,1.188979,85,20,9,11,NeuronType.PYRAMIDAL
68,50.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,1.316970,0.899547,-0.417424,24.755732,10.928132,-13.827600,13.036351,5.913839,-7.122512,66,34,14,20,NeuronType.PYRAMIDAL
54,56.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,1.017541,0.737775,-0.279766,10.921520,8.402061,-2.519458,5.969531,4.569918,-1.399612,52,44,25,19,NeuronType.PYRAMIDAL
34,56.563733,True,92.563733,True,False,SplitPartitionMembership.SHARED,1.181583,1.269171,0.087588,14.207869,21.312142,7.104274,7.694726,11.290657,3.595931,32,46,18,28,NeuronType.PYRAMIDAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,10.854761,10.494782,-0.359979,16.270101,15.489464,-0.780637,13.562431,12.992123,-0.570308,101,262,144,118,NeuronType.CONTAMINATED
104,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,9.607706,9.436767,-0.170940,50.823551,49.491147,-1.332404,30.215629,29.463957,-0.751672,102,363,197,166,NeuronType.CONTAMINATED
105,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,0.129664,0.146140,0.016476,6.439696,7.921885,1.482188,3.284680,4.034012,0.749332,103,9,5,4,NeuronType.PYRAMIDAL
106,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,12.817478,13.180032,0.362554,47.537618,50.207624,2.670006,30.177548,31.693828,1.516280,104,359,196,163,NeuronType.CONTAMINATED


In [36]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
@define(slots=False)
class TrackPartitioningResult:
	is_S_pf_only: np.ndarray
	is_S_only: np.ndarray
	S_only_aclus: np.ndarray
	S_only_df: pd.DataFrame

# Four distinct subgroups are formed:  pf on neither, pf on both, pf on only long, pf on only short
# L_only_aclus, S_only_aclus

In [37]:
## 2023-05-19 - Get S-only pfs
is_S_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_long_pf']), neuron_replay_stats_df['has_short_pf'])
is_S_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY
assert (is_S_pf_only == is_S_only).all()
S_only_aclus = neuron_replay_stats_df.index[is_S_only].to_numpy()
neuron_replay_stats_df[is_S_pf_only]

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
4,NaN,False,178.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,0.100774,0.760024,0.659250,NaN,14.188911,NaN,NaN,7.474468,NaN,2,61,0,61,NeuronType.PYRAMIDAL
13,NaN,False,220.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,0.376134,0.613902,0.237768,6.815635,8.249296,1.433661,3.595885,4.431599,0.835714,11,20,6,14,NeuronType.PYRAMIDAL
21,NaN,False,100.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,0.314844,0.392003,0.077159,7.886320,7.739596,-0.146724,4.100582,4.065800,-0.034782,19,31,13,18,NeuronType.PYRAMIDAL
23,NaN,False,116.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,0.605492,0.937648,0.332156,7.249651,11.013565,3.763914,3.927571,5.975606,2.048035,21,18,4,14,NeuronType.PYRAMIDAL
52,NaN,False,200.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,1.377775,1.367204,-0.010572,11.136869,14.971473,3.834604,6.257322,8.169338,1.912016,50,74,19,55,NeuronType.PYRAMIDAL
58,NaN,False,174.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,0.348962,0.938830,0.589868,7.011756,12.335964,5.324209,3.680359,6.637397,2.957038,56,60,9,51,NeuronType.PYRAMIDAL
67,NaN,False,206.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,0.877291,1.555967,0.678676,8.505614,8.947704,0.442090,4.691453,5.251835,0.560383,65,51,19,32,NeuronType.PYRAMIDAL
70,NaN,False,172.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,1.087842,0.807833,-0.280009,7.137189,6.963502,-0.173687,4.112515,3.885668,-0.226848,68,33,8,25,NeuronType.PYRAMIDAL
81,NaN,False,182.563733,True,True,SplitPartitionMembership.RIGHT_ONLY,0.985040,0.812764,-0.172277,8.963245,6.764478,-2.198767,4.974143,3.788621,-1.185522,79,46,13,33,NeuronType.PYRAMIDAL


In [38]:
## Show L-only pfs stop replaying on S
is_L_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_short_pf']), neuron_replay_stats_df['has_long_pf'])
is_L_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY
assert (is_L_pf_only == is_L_only).all()
L_only_aclus = neuron_replay_stats_df.index[is_L_only].to_numpy()
neuron_replay_stats_df[is_L_only]


## For ('kdiba', 'gor01', 'one', '2006-6-09_1-22-43') - Have L-only cells [24, 98] that have ['short_num_replays'] = [8, 7]. We were hoping that there would be few to no replays on the S-track that involved L-only cells.

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
2,212.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.305475,0.075055,-0.230420,10.361654,NaN,NaN,5.333564,NaN,NaN,0,26,26,0,NeuronType.PYRAMIDAL
8,208.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.635524,0.485675,-0.149849,9.130620,8.520738,-0.609882,4.883072,4.503206,-0.379865,6,30,10,20,NeuronType.PYRAMIDAL
19,252.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.310495,1.077981,0.767486,16.505982,19.120475,2.614493,8.408239,10.099228,1.690989,17,28,12,16,NeuronType.PYRAMIDAL
28,250.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,1.423936,1.208580,-0.215356,8.121781,20.622364,12.500583,4.772859,10.915472,6.142614,26,3,1,2,NeuronType.PYRAMIDAL
36,252.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.966328,1.122273,0.155945,19.282126,23.696671,4.414545,10.124227,12.409472,2.285245,34,93,46,47,NeuronType.PYRAMIDAL
54,56.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,1.017541,0.737775,-0.279766,10.921520,8.402061,-2.519458,5.969531,4.569918,-1.399612,52,44,25,19,NeuronType.PYRAMIDAL
63,86.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,2.011771,1.364999,-0.646773,9.121347,10.441166,1.319818,5.566559,5.903082,0.336523,61,118,56,62,NeuronType.PYRAMIDAL
68,50.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,1.316970,0.899547,-0.417424,24.755732,10.928132,-13.827600,13.036351,5.913839,-7.122512,66,34,14,20,NeuronType.PYRAMIDAL
73,216.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.465649,0.441438,-0.024211,11.883865,14.105492,2.221627,6.174757,7.273465,1.098708,71,48,33,15,NeuronType.PYRAMIDAL
74,60.563733,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,1.487999,0.528636,-0.959363,12.713187,9.942147,-2.771040,7.100593,5.235392,-1.865202,72,69,37,32,NeuronType.PYRAMIDAL


In [39]:
## 2023-05-23 - Get Common (SHARED) placefields
## Goal 1: From the cells with the placefields on both tracks, compute the degree to which they remap in position and sort them according to their distance.
is_BOTH_pf_only = np.logical_and(neuron_replay_stats_df['has_short_pf'], neuron_replay_stats_df['has_long_pf']) # (63,)
BOTH_pf_only_aclus = neuron_replay_stats_df.index[is_BOTH_pf_only].to_numpy()

## NOTE: is_BOTH_pf_only is a much more stringent requirement (and a strict subset) than `is_BOTH_only`
is_BOTH_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED # (99,)
BOTH_only_aclus = neuron_replay_stats_df.index[is_BOTH_only].to_numpy()
assert BOTH_only_aclus.shape[0] >= BOTH_pf_only_aclus.shape[0]

BOTH_pf_only_df = neuron_replay_stats_df[is_BOTH_pf_only].copy()
BOTH_pf_only_df['long_short_pf_peak_x_displacement'] = BOTH_pf_only_df['long_pf_peak_x'].values - BOTH_pf_only_df['short_pf_peak_x'].values
BOTH_pf_only_df['long_short_pf_peak_x_distance'] = BOTH_pf_only_df['long_short_pf_peak_x_displacement'].abs()
BOTH_pf_only_df.sort_values(by=['long_short_pf_peak_x_distance'], inplace=True, ascending=False)
BOTH_pf_only_df

#TODO 2023-05-23 - Can do more detailed peaks analysis with: long_results.RatemapPeaksAnalysis and short_results.RatemapPeaksAnalysis



,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,...,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type,long_short_pf_peak_x_displacement,long_short_pf_peak_x_distance
86,64.563733,True,178.563733,True,False,SplitPartitionMembership.SHARED,2.188722,1.416173,-0.772548,11.203227,...,6.695974,6.350041,-0.345933,84,152,76,76,NeuronType.PYRAMIDAL,-114.0,114.0
61,202.563733,True,96.563733,True,False,SplitPartitionMembership.SHARED,1.811136,1.906058,0.094921,9.523538,...,5.667337,6.046077,0.378740,59,132,79,53,NeuronType.PYRAMIDAL,106.0,106.0
62,258.563733,True,160.563733,True,False,SplitPartitionMembership.SHARED,2.736108,1.949009,-0.787099,17.909160,...,10.322634,10.240643,-0.081992,60,183,90,93,NeuronType.PYRAMIDAL,98.0,98.0
57,258.563733,True,180.563733,True,False,SplitPartitionMembership.SHARED,1.501704,1.360496,-0.141208,10.275277,...,5.888490,5.571848,-0.316642,55,91,44,47,NeuronType.PYRAMIDAL,78.0,78.0
41,214.563733,True,136.563733,True,False,SplitPartitionMembership.SHARED,1.046637,0.728784,-0.317854,13.322294,...,7.184466,6.096120,-1.088346,39,171,95,76,NeuronType.PYRAMIDAL,78.0,78.0
11,258.563733,True,182.563733,True,False,SplitPartitionMembership.SHARED,0.746574,0.909667,0.163094,14.571280,...,7.658927,6.316646,-1.342281,9,61,29,32,NeuronType.PYRAMIDAL,76.0,76.0
3,144.563733,True,198.563733,True,False,SplitPartitionMembership.SHARED,0.257163,1.096455,0.839293,13.312611,...,6.784887,5.384832,-1.400054,1,64,18,46,NeuronType.PYRAMIDAL,-54.0,54.0
102,36.563733,True,88.563733,True,False,SplitPartitionMembership.SHARED,1.667443,1.431349,-0.236095,7.706498,...,4.686970,4.781130,0.094159,100,39,20,19,NeuronType.PYRAMIDAL,-52.0,52.0
53,218.563733,True,168.563733,True,False,SplitPartitionMembership.SHARED,1.027919,0.813517,-0.214402,11.540984,...,6.284452,5.209091,-1.075360,51,123,66,57,NeuronType.PYRAMIDAL,50.0,50.0
75,208.563733,True,160.563733,True,False,SplitPartitionMembership.SHARED,2.152696,2.603371,0.450675,21.457308,...,11.805002,14.123231,2.318229,73,153,80,73,NeuronType.PYRAMIDAL,48.0,48.0


In [46]:
# 2023-06-19 - Look at the Long and Short only aclus and find replays in which they replay
# L_only_aclus, S_only_aclus

# long_replays

exclusive_aclus = np.hstack((L_only_aclus, S_only_aclus))
exclusive_aclus
# find the cells that fire in each replay. I think this is already computed/used somewhere


array([  2,   8,  19,  28,  36,  54,  63,  68,  73,  74, 109,   4,  13,
        21,  23,  52,  58,  67,  70,  81], dtype=int64)

In [48]:
# approach copied from `pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations._epoch_unit_avg_firing_rates`

included_neuron_ids = deepcopy(exclusive_aclus)
spikes_df = deepcopy(curr_active_pipeline.sess.spikes_df)
filter_epochs = deepcopy(curr_active_pipeline.sess.replay)

if included_neuron_ids is None:
	included_neuron_ids = spikes_df.spikes.neuron_ids

if isinstance(filter_epochs, pd.DataFrame):
	filter_epochs_df = filter_epochs
else:
	filter_epochs_df = filter_epochs.to_dataframe()
	
filter_epoch_spikes_df = deepcopy(spikes_df)

In [ ]:
# first slice only to the neurons of interest:
# filter_epoch_spikes_df


In [ ]:


for epoch_id in np.arange(np.shape(filter_epochs_df)[0]):
	epoch_start = filter_epochs_df.start.values[epoch_id]
	epoch_end = filter_epochs_df.stop.values[epoch_id]
	epoch_spikes_df = spikes_df.spikes.time_sliced(t_start=epoch_start, t_stop=epoch_end)
	# epoch_spikes_df = filter_epoch_spikes_df[filter_epoch_spikes_df['temp_epoch_id'] == epoch_id]
	for aclu, unit_epoch_spikes_df in zip(included_neuron_ids, epoch_spikes_df.spikes.get_split_by_unit(included_neuron_ids=included_neuron_ids)):
		if aclu not in epoch_avg_firing_rate:
			epoch_avg_firing_rate[aclu] = []
		epoch_avg_firing_rate[aclu].append((float(np.shape(unit_epoch_spikes_df)[0]) / (epoch_end - epoch_start)))


In [ ]:
""" 24 cells, 133 Epochs, 789 Total Timebins
# Other
	
	- is_non_firing_time_bin: numpy.ndarray - (24, 789)
	- all_epochs_num_epoch_time_bins: numpy.ndarray - (133,)
	
    ## Indexing Helpers:
		- all_epochs_reverse_flat_epoch_indicies_array: numpy.ndarray - (789,)
		- split_by_epoch_reverse_flattened_time_bin_indicies: list - (133,)
    
	- original_1D_decoder: pyphoplacecellanalysis.Analysis.Decoder.reconstruction.BasePositionDecoder
		- pf: neuropy.analyses.placefields.PfND
		- neuron_IDXs: numpy.ndarray - (24,)
		- neuron_IDs: numpy.ndarray - (24,)
		- F: numpy.ndarray - (119, 24)
		- P_x: numpy.ndarray - (119, 1)
        
	- flat_all_epochs_decoded_epoch_time_bins: numpy.ndarray - (24, 789)

# Measured
	- flat_all_epochs_measured_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - (24, 789)


# Expected
	- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_firing_rates: numpy.ndarray - (24, 789)
    
    
    
## Epoch-based
	- all_epochs_decoded_epoch_time_bins_mean: numpy.ndarray - (133, 24)

    
    
    
- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - .shape: (24, 789) 
- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - .shape: (24, 789)


"""

# 2023-04-13 - Find Good looking epochs:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.patheffects as path_effects

from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

curr_results_obj = long_results_obj
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_pagination_controller = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(curr_results_obj.active_filter_epochs, curr_results_obj.all_included_filter_epochs_decoder_result, 
	xbin=curr_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='TestDecodedEpochSlicesPaginationController', active_context=active_identifying_session_ctx,  max_subplots_per_page=200) # 10
# _out_pagination_controller

In [11]:
## Getting selections out of DecoedEpochSlicesPaginationController
curr_active_pipeline.display_output_history_list
# [IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'long_only', '[2, 8, 19, 28, 36, 54, 63, 68, 73, 74, 109]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'short_only', '[4, 13, 21, 23, 52, 58, 67, 70, 81]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '1of9', '[3, 5, 6, 7, 9, 10, 11, 12, 14, 15]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '2of9', '[16, 17, 18, 20, 22, 24, 25, 26, 27, 29]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '3of9', '[30, 31, 32, 33, 34, 35, 37, 38, 39, 40]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '4of9', '[41, 42, 43, 44, 45, 46, 47, 48, 49, 50]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '5of9', '[51, 53, 55, 56, 57, 59, 60, 61, 62, 64]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '6of9', '[65, 66, 69, 71, 72, 75, 76, 77, 78, 79]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '7of9', '[80, 82, 83, 84, 85, 86, 87, 88, 89, 90]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '8of9', '[91, 92, 93, 94, 95, 96, 97, 98, 99, 100]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'BatchPhoJonathanReplayFRC', 'shared', '9of9', '[101, 102, 103, 104, 105, 106, 107, 108]')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_1d_placefields')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_1d_placefield_validations')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_2d_placefield_result_plot_ratemaps_2D')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze1', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze2', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'maze', '_display_2d_placefield_occupancy')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_short_long_firing_rate_index_comparison')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_short_expected_v_observed_firing_rate')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_grid_bin_bounds_validation')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_running_and_replay_speeds_over_time')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_short_laps')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_and_short_firing_rate_replays_v_laps')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_short_long_pf1D_comparison')>,
#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', '_display_long_and_short_stacked_epoch_slices')>]

# Ideally could search like: '_display_long_and_short_stacked_epoch_slices'
active_result_backup = curr_active_pipeline.last_added_display_output

In [16]:
list(active_result_backup.keys()) # ['name', 'context', 'figures', 'axes', 'plot_data']

active_result_backup['context'] # (IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'long_results_obj')>,
								#  IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15', 'DecodedEpochSlices', 'replays', 'short_results_obj')>)

pagination_controllers = active_result_backup.plot_data['controllers']

DecodedEpochSlicesPaginatedFigureController(params=VisualizationParameters({'name': 'TestDecodedEpochSlicesPaginationController', 'window_title': 'Stacked Epoch Slices View - MATPLOTLIB subplots Version', 'num_slices': 122, '_debug_test_max_num_slices': 200, 'active_num_slices': 122, 'global_epoch_start_t': 44.58873621060047, 'global_epoch_end_t': 2045.3932618786348, 'single_plot_fixed_height': 100.0, 'all_plots_height': 12200.0, 'epoch_labels': ['epoch[0]', 'epoch[1]', 'epoch[2]', 'epoch[3]', 'epoch[4]', 'epoch[5]', 'epoch[6]', 'epoch[7]', 'epoch[8]', 'epoch[9]', 'epoch[10]', 'epoch[11]', 'epoch[12]', 'epoch[13]', 'epoch[14]', 'epoch[15]', 'epoch[16]', 'epoch[17]', 'epoch[18]', 'epoch[19]', 'epoch[20]', 'epoch[21]', 'epoch[22]', 'epoch[23]', 'epoch[24]', 'epoch[25]', 'epoch[26]', 'epoch[27]', 'epoch[28]', 'epoch[29]', 'epoch[30]', 'epoch[31]', 'epoch[32]', 'epoch[33]', 'epoch[34]', 'epoch[35]', 'epoch[36]', 'epoch[37]', 'epoch[38]', 'epoch[39]', 'epoch[40]', 'epoch[41]', 'epoch[42]', 

In [17]:
long_controller = pagination_controllers[0]
short_controller = pagination_controllers[1]

In [18]:
long_controller.selected_indicies # array([ 13,  14,  15,  25,  27,  28,  31,  37,  42,  45,  48,  57,  61,
        # 62,  63,  76,  79,  82,  89,  90, 111, 112, 113, 115])

array([ 13,  14,  15,  25,  27,  28,  31,  37,  42,  45,  48,  57,  61,
        62,  63,  76,  79,  82,  89,  90, 111, 112, 113, 115])

In [19]:
short_controller.selected_indicies # array([  9,  11,  13,  14,  15,  20,  22,  25,  37,  40,  45,  48,  61,
        # 62,  76,  79,  84,  89,  90,  93,  94, 111, 112, 113, 115, 121])

array([  9,  11,  13,  14,  15,  20,  22,  25,  37,  40,  45,  48,  61,
        62,  76,  79,  84,  89,  90,  93,  94, 111, 112, 113, 115, 121])

In [28]:
assert long_controller.is_selected.shape[0] == long_results_obj.active_filter_epochs.n_epochs
assert short_controller.is_selected.shape[0] == short_results_obj.active_filter_epochs.n_epochs
# for some reason the active_filter_epochs on both the long and the short objects contain properties about the other one as well, so keep with tradition and add them to both as well
long_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
long_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected
short_results_obj.active_filter_epochs._df['long_is_user_included'] = long_controller.is_selected
short_results_obj.active_filter_epochs._df['short_is_user_included'] = short_controller.is_selected

In [20]:
long_controller.selected_epoch_times

array([[ 193.64762949,  193.89308217],
       [ 218.17630813,  218.37116499],
       [ 267.60596091,  267.7736613 ],
       [ 419.65138962,  419.87589126],
       [ 434.1020395 ,  434.31575843],
       [ 449.6101634 ,  449.82833673],
       [ 689.08064303,  689.28462373],
       [ 799.94018819,  800.13956089],
       [ 857.29808522,  857.47764353],
       [ 867.65532792,  867.83623791],
       [ 906.06778057,  906.26859711],
       [1077.0301148 , 1077.22162318],
       [1152.56290685, 1152.76126579],
       [1158.83731004, 1159.07892271],
       [1191.68010666, 1191.88958623],
       [1318.30751319, 1318.50046541],
       [1368.63077958, 1368.84652603],
       [1424.49921818, 1424.71370511],
       [1485.88785363, 1486.14562501],
       [1493.48502862, 1493.69324867],
       [1835.03884542, 1835.21480949],
       [1848.99152436, 1849.22361384],
       [1866.80939171, 1867.07201686],
       [1892.26771852, 1892.51793279]])

In [21]:
short_controller.selected_epoch_times

array([[ 181.66634411,  181.8681744 ],
       [ 188.79725118,  189.01570098],
       [ 193.64762949,  193.89308217],
       [ 218.17630813,  218.37116499],
       [ 267.60596091,  267.7736613 ],
       [ 338.42538577,  338.64429637],
       [ 367.85316462,  368.12417632],
       [ 419.65138962,  419.87589126],
       [ 799.94018819,  800.13956089],
       [ 815.26159603,  815.46843368],
       [ 867.65532792,  867.83623791],
       [ 906.06778057,  906.26859711],
       [1152.56290685, 1152.76126579],
       [1158.83731004, 1159.07892271],
       [1318.30751319, 1318.50046541],
       [1368.63077958, 1368.84652603],
       [1454.74685197, 1454.96831234],
       [1485.88785363, 1486.14562501],
       [1493.48502862, 1493.69324867],
       [1530.54057956, 1530.78093272],
       [1540.99904461, 1541.24124096],
       [1835.03884542, 1835.21480949],
       [1848.99152436, 1849.22361384],
       [1866.80939171, 1867.07201686],
       [1892.26771852, 1892.51793279],
       [2045.09917947, 20

In [24]:
long_results_obj.active_filter_epochs.n_epochs # 122

122

# TODO 2023-06-19 14:31: - [ ] Come up with system for storing user-annotations:
 Need to store user-annotations and labeled selections for Epochs:

	date_annotated: datetime - the date the annotations were made
	annotation_author: str - the person who made the annotations
	annotation_content: Any - the content of the annotation



In [ ]:
# All epochs are shown for both decoders. So the result should be an epochs table with separate columns for the inclusion in each decoder.




# Other Programmatic Figures

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

## Builds proper sort indicies for '_display_short_long_pf1D_comparison'
long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap

curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])
curr_any_context_neurons

In [ ]:
from neuropy.plotting.ratemaps import plot_ratemap_1D
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_short_v_long_pf1D_comparison, LongShortTrackComparingDisplayFunctions

curr_active_pipeline.reload_default_display_functions()

%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None
# sort_idx = None
# sort_idx = sort_ind
# sort_idx = sort_indicies
# sort_idx = new_all_aclus_sort_indicies
sort_idx = 'peak_long'
# sort_idx = 'bad'

# out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=True, fignum='Short v Long pf1D Comparison',
#                                    long_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None, 'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
#                                    short_kwargs={'included_unit_neuron_IDs': curr_any_context_neurons, 'included_unit_indicies': None,'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
#                                   )

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )
                                  

# ax = out.axes[0]

In [ ]:
curr_active_pipeline.display('_display_short_long_pf1D_comparison', single_figure=False)

# 2023-06-01 - Testing new metrics:

In [ ]:

curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
## Extract variables from results object:
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

print(f'long_test: {np.mean(long_long_diff)}')
print(f'short_test: {np.mean(short_short_diff)}')

In [ ]:
long_session.replay

In [ ]:
# long_results_obj.original_1D_decoder
replay_result_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
replay_result_df

In [ ]:
short_results_obj.active_filter_epochs.to_dataframe()

In [ ]:
# Look at lap speed over time - doesn't look like there's a difference between the long and the short track speeds
global_session.position.compute_higher_order_derivatives()
running_pos_df = global_session.position.to_dataframe()
ax = running_pos_df.plot(x='t', y=['lin_pos','speed'], title='Running Speed over Time')
ax

In [ ]:
# Call the function
fig, ax = plot_lines(replay_result_df)

In [ ]:
import matplotlib
%matplotlib qt

curr_active_pipeline.reload_default_display_functions()
out_man = curr_active_pipeline.get_output_manager()
out_man.get_figure_output_path
curr_active_pipeline.write_figure_to_output_path



In [ ]:
_out = curr_active_pipeline.display('_display_grid_bin_bounds_validation')
# _display_running_and_replay_speeds_over_time


# 2023-06-13 11:01: - [ ] New display function testing

In [ ]:
import matplotlib as mpl
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path, programmatic_display_to_PDF
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_metadata_from_display_context # newer version of build_pdf_export_metadata
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

%matplotlib qt

from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file

In [ ]:
curr_active_pipeline.reload_default_display_functions()


In [ ]:
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap, visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_raster_plot # used in `plot_kourosh_activity_style_figure`
import pyphoplacecellanalysis.External.pyqtgraph as pg # used in `plot_kourosh_activity_style_figure`

heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(long_pf1D.ratemap.normalized_tuning_curves, title=f"pf1D Visualization")
# Apply the colormap
# heatmap_pf1D_img.setLookupTable(lut)
heatmap_pf1D_win.show()

In [ ]:
heatmap_pf1D_win.close()

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

## Builds proper sort indicies for '_display_short_long_pf1D_comparison'
long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap

curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])
curr_any_context_neurons

In [ ]:
from neuropy.utils.mixins.time_slicing import add_epochs_id_identity

spikes_df = deepcopy(long_session.spikes_df)
filter_epochs = deepcopy(long_replays)
shared_aclus = curr_any_context_neurons # all aclus to use
debug_print = True

if isinstance(filter_epochs, pd.DataFrame):
	filter_epochs_df = filter_epochs
else:
	filter_epochs_df = filter_epochs.to_dataframe()
	
if debug_print:
	print(f'filter_epochs: {filter_epochs.epochs.n_epochs}')
## Get the spikes during these epochs to attempt to decode from:
filter_epoch_spikes_df = deepcopy(spikes_df)
filter_epoch_spikes_df = filter_epoch_spikes_df[filter_epoch_spikes_df['aclu'].isin(shared_aclus)] ## restrict to only the shared aclus for both short and long
filter_epoch_spikes_df, _temp_neuron_id_to_new_IDX_map = filter_epoch_spikes_df.spikes.rebuild_fragile_linear_neuron_IDXs() # I think this must be done prior to restricting to the current epoch, but after restricting to the shared_aclus

## Add the epoch ids to each spike so we can easily filter on them:
filter_epoch_spikes_df = add_epochs_id_identity(filter_epoch_spikes_df, filter_epochs_df, epoch_id_key_name='temp_epoch_id', epoch_label_column_name=None, no_interval_fill_value=-1)
if debug_print:
	print(f'np.shape(filter_epoch_spikes_df): {np.shape(filter_epoch_spikes_df)}')
filter_epoch_spikes_df = filter_epoch_spikes_df[filter_epoch_spikes_df['temp_epoch_id'] != -1] # Drop all non-included spikes
if debug_print:
	print(f'np.shape(filter_epoch_spikes_df): {np.shape(filter_epoch_spikes_df)}')

# returns `filter_epoch_spikes_df`
filter_epoch_spikes_df

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_raster_plot # used in `plot_kourosh_activity_style_figure`
from pyphocorehelpers.DataStructure.RenderPlots.PyqtgraphRenderPlots import PyqtgraphRenderPlots
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _plot_empty_raster_plot_frame
import pyphoplacecellanalysis.External.pyqtgraph as pg
from pyphoplacecellanalysis.External.pyqtgraph.Qt import QtGui # for QColor build_neurons_color_data
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import UnitSortOrderManager, RasterPlotParams, RasterScatterPlotManager, _add_spikes_df_visualization_columns

# ## Create the raster plot for the replay:
# app, win, plots, plots_data = plot_raster_plot(_active_epoch_spikes_df, shared_aclus, scatter_app_name=f"Raster Epoch[{epoch_idx}]")
app, win, plots, plots_data = _plot_empty_raster_plot_frame(scatter_app_name="Pho Stacked Replays", defer_show=False)
callout_glw = win.addLayout()
row_start_idx = 0
# plots = PyqtgraphRenderPlots('plots', layout=None, posteriors=[], placefields=[])

plots.layout = callout_glw
plots.ax = {}
plots_data.all_spots_dict = {}

neuron_ids = deepcopy(shared_aclus)
n_cells = len(shared_aclus)
fragile_linear_neuron_IDXs = np.arange(n_cells)
unit_sort_order = np.arange(n_cells) # in-line sort order
params = RasterPlotParams()
params.build_neurons_color_data(fragile_linear_neuron_IDXs=fragile_linear_neuron_IDXs)
manager = UnitSortOrderManager(neuron_ids=neuron_ids, fragile_linear_neuron_IDXs=fragile_linear_neuron_IDXs, n_cells=n_cells, unit_sort_order=unit_sort_order, params=params)
manager.update_series_identity_y_values()
raster_plot_manager = RasterScatterPlotManager(unit_sort_manager=manager)
raster_plot_manager._build_cell_configs()
# Update the dataframe
filter_epoch_spikes_df = _add_spikes_df_visualization_columns(manager, filter_epoch_spikes_df)

## Build the individual epoch raster plot rows:
for an_epoch in filter_epochs_df.itertuples():
	# print(f'an_epoch: {an_epoch}')
	if an_epoch.Index < 10:
		_active_epoch_spikes_df = filter_epoch_spikes_df[filter_epoch_spikes_df['temp_epoch_id'] == an_epoch.Index]
		_active_plot_title: str = f"Epoch[{an_epoch.label}]"
		
		## Create the raster plot for the replay:
		# if win is None:
		# 	# Initialize
		# 	app, win, plots, plots_data = plot_raster_plot(_active_epoch_spikes_df, shared_aclus, scatter_app_name=f"Raster Epoch[{an_epoch.label}]")
		# else:
		# add a new row
		new_ax = plots.layout.addPlot(row=int(an_epoch.Index), col=0)
		spots = Render2DScrollWindowPlotMixin.build_spikes_all_spots_from_df(_active_epoch_spikes_df, raster_plot_manager.config_fragile_linear_neuron_IDX_map)
		plots_data.all_spots_dict[an_epoch.Index] = spots

		# Common Tick Label
		vtick = QtGui.QPainterPath()

		# Thicker Tick Label:
		tick_width = 0.1
		half_tick_width = 0.5 * tick_width
		vtick.moveTo(-half_tick_width, -0.5)
		vtick.addRect(-half_tick_width, -0.5, tick_width, 1.0) # x, y, width, height

		scatter_plot = pg.ScatterPlotItem(name='spikeRasterOverviewWindowScatterPlotItem', pxMode=True, symbol=vtick, size=10, pen={'color': 'w', 'width': 1})
		scatter_plot.setObjectName(f'scatter_plot_{_active_plot_title}') # this seems necissary, the 'name' parameter in addPlot(...) seems to only change some internal property related to the legend AND drastically slows down the plotting
		scatter_plot.opts['useCache'] = True
		scatter_plot.addPoints(plots_data.all_spots_dict[an_epoch.Index]) # , hoverable=True
		new_ax.addItem(scatter_plot)
		new_ax.setXRange(an_epoch.start, an_epoch.end)
		new_ax.setYRange(0, len(shared_aclus)-1)
		# Disable Interactivity
		new_ax.setMouseEnabled(x=False, y=False)
		new_ax.setMenuEnabled(False)
		plots.ax[an_epoch.Index] = new_ax
		
	

In [ ]:
an_epoch.start, an_epoch.end

In [ ]:
long_replays

_active_epoch_spikes_df = deepcopy(long_session.spikes_df)
# _active_epoch_spikes_df = long_results_obj.original_1D_decoder.spikes_df.copy()

_active_epoch_spikes_df = _active_epoch_spikes_df[_active_epoch_spikes_df['aclu'].isin(shared_aclus)] ## restrict to only the shared aclus for both short and long
_active_epoch_spikes_df, _temp_neuron_id_to_new_IDX_map = _active_epoch_spikes_df.spikes.rebuild_fragile_linear_neuron_IDXs() # I think this must be done prior to restricting to the current epoch, but after restricting to the shared_aclus
_active_epoch_spikes_df = _active_epoch_spikes_df.spikes.time_sliced(*active_epoch[0]) # restrict to the active epoch